# Get all parks and playgrounds in Victoria

In [1]:
%cd ../scripts 
from nearbyFeatures import getAllFeatures
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, shape

/mnt/c/Users/ohanr/OneDrive/Unimelb/3rd Year/MAST30034/generic-real-estate-consulting-project-group-25/scripts


## Getting parks and playgrounds

In [2]:
parks = getAllFeatures(key="leisure", value="park")
playgrounds = getAllFeatures(key="leisure", value="playground")

Saving the results in csv

In [13]:
df_park = pd.DataFrame()
name = []
lat = []
lng = []
# For each park found by the api, we extract the name and coordinates to save it in a new csv file 
for park in parks["elements"]:
    try: 
        name.append(park["tags"]["name"])
    except KeyError:
        name.append(park["id"])
    lat.append(park["lat"])
    lng.append(park["lon"])

df_park["Park_Name"] = name
df_park["Latitude"] = lat
df_park["Longitude"] = lng
df_park.to_csv("../data/raw/parks.csv", index=False)


In [15]:
df_playground = pd.DataFrame()
name = []
lat = []
lng = []
# For each playgrounds found by the api, we extract the name and coordinates to save it in a new csv file 
for playground in playgrounds["elements"]:
    try: 
        name.append(playground["tags"]["name"])
    except KeyError:
        name.append(playground["id"])
    lat.append(playground["lat"])
    lng.append(playground["lon"])

df_playground["Playground_Name"] = name
df_playground["Latitude"] = lat
df_playground["Longitude"] = lng
df_playground.to_csv("../data/raw/playground.csv", index=False)

## Assigning suburbs to parks and playgrounds

In [14]:
playgrounds = pd.read_csv("../data/raw/playground.csv")
parks = pd.read_csv("../data/raw/parks.csv")
suburb = gpd.read_file("../data/raw/Suburb Shapes/vic_localities.shp")[["LOC_NAME", "geometry"]]
candid_suburb = pd.read_csv("../data/raw/sub_4_pred.csv")

In [15]:
# getting list of wanted suburbs into a clean format
suburb_list = []
for row, sub in candid_suburb.iterrows():
    subs = sub["Suburb"].split(", ")
    n = len(subs)
    for i in range(0, n):
        suburb_list.append([subs[i], sub["Postcode"]])
clean_suburbs = pd.DataFrame(suburb_list, columns=["Suburb", "Postcode"])
clean_suburbs = clean_suburbs.drop_duplicates()


In [19]:
# Only extract the wanted suburbs from the list of entire suburbs in VIC
suburb_joined = suburb.merge(clean_suburbs, how = "inner", left_on = "LOC_NAME", right_on = "Suburb")

,LOC_NAME,geometry,Suburb,Postcode
0,Aberfeldie,"POLYGON ((144.89830 -37.76464, 144.89790 -37.7...",Aberfeldie,3040
1,Albanvale,"POLYGON ((144.76355 -37.75072, 144.76338 -37.7...",Albanvale,3021
2,Albion,"POLYGON ((144.82793 -37.78180, 144.82781 -37.7...",Albion,3020
3,Alphington,"POLYGON ((145.02092 -37.78696, 145.02066 -37.7...",Alphington,3078
4,Altona,"POLYGON ((144.79085 -37.87025, 144.79111 -37.8...",Altona,3018


In [20]:
# If the coordinate point represented by the playground lies within the boundary of a suburb, 
# Increment the number of playgrounds in a suburb
suburb_joined["NumPlaygrounds"] = 0
for i, playg in playgrounds.iterrows():
    play_point = Point(playg["Longitude"], playg["Latitude"]) 
    for j, sub in suburb_joined.iterrows():
        sub_geom = shape(sub["geometry"])
        if sub_geom.contains(play_point):
            suburb_joined["NumPlaygrounds"][j] += 1
            break
suburb_joined

/tmp/ipykernel_803/208984597.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suburb_joined["NumPlaygrounds"][j] += 1


,LOC_NAME,geometry,Suburb,Postcode,NumPlaygrounds
0,Aberfeldie,"POLYGON ((144.89830 -37.76464, 144.89790 -37.7...",Aberfeldie,3040,0
1,Albanvale,"POLYGON ((144.76355 -37.75072, 144.76338 -37.7...",Albanvale,3021,0
2,Albion,"POLYGON ((144.82793 -37.78180, 144.82781 -37.7...",Albion,3020,1
3,Alphington,"POLYGON ((145.02092 -37.78696, 145.02066 -37.7...",Alphington,3078,3
4,Altona,"POLYGON ((144.79085 -37.87025, 144.79111 -37.8...",Altona,3018,0
...,...,...,...,...,...
203,Williamstown,"POLYGON ((144.90359 -37.86100, 144.90360 -37.8...",Williamstown,3016,4
204,Williamstown North,"POLYGON ((144.86270 -37.85509, 144.85778 -37.8...",Williamstown North,3016,1
205,Windsor,"POLYGON ((145.00210 -37.85496, 145.00199 -37.8...",Windsor,3181,3
206,Woori Yallock,"POLYGON ((145.56403 -37.79780, 145.56412 -37.7...",Woori Yallock,3139,1


In [21]:
# If the coordinate point represented as the park lies within the boundary of a suburb, 
# Increment the number of parks in a suburb
suburb_joined["NumParks"] = 0
for i, park in parks.iterrows():
    park_point = Point(park["Longitude"], park["Latitude"]) 
    for j, sub in suburb_joined.iterrows():
        sub_geom = shape(sub["geometry"])
        if sub_geom.contains(park_point):
            suburb_joined["NumParks"][j] += 1
            break
suburb_joined

/tmp/ipykernel_803/2748614409.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suburb_joined["NumParks"][j] += 1


,LOC_NAME,geometry,Suburb,Postcode,NumPlaygrounds,NumParks
0,Aberfeldie,"POLYGON ((144.89830 -37.76464, 144.89790 -37.7...",Aberfeldie,3040,0,0
1,Albanvale,"POLYGON ((144.76355 -37.75072, 144.76338 -37.7...",Albanvale,3021,0,0
2,Albion,"POLYGON ((144.82793 -37.78180, 144.82781 -37.7...",Albion,3020,1,0
3,Alphington,"POLYGON ((145.02092 -37.78696, 145.02066 -37.7...",Alphington,3078,3,0
4,Altona,"POLYGON ((144.79085 -37.87025, 144.79111 -37.8...",Altona,3018,0,0
...,...,...,...,...,...,...
203,Williamstown,"POLYGON ((144.90359 -37.86100, 144.90360 -37.8...",Williamstown,3016,4,0
204,Williamstown North,"POLYGON ((144.86270 -37.85509, 144.85778 -37.8...",Williamstown North,3016,1,0
205,Windsor,"POLYGON ((145.00210 -37.85496, 145.00199 -37.8...",Windsor,3181,3,0
206,Woori Yallock,"POLYGON ((145.56403 -37.79780, 145.56412 -37.7...",Woori Yallock,3139,1,0


In [22]:
suburb_joined[["Suburb", "Postcode", "NumPlaygrounds", "NumParks"]].to_csv("../data/curated/suburb_w_leisure.csv", index = False)